In [1]:
from __future__ import print_function

%matplotlib inline

import sys
import tensorflow as tf
from collections import OrderedDict

import numpy as np
import scipy.misc
from input import *
from model import *
from train import *
import matplotlib.pyplot as plt
tf.reset_default_graph()

1.3.0


In [2]:
image_shape = (160,576)
iterator, filename = get_train_inputs(batch_size=100,repeat=False,num_classes=2, image_shape=image_shape)

In [3]:
_show = False
if _show:
    with tf.Session() as sess:
        sess.run(iterator.initializer, feed_dict={filename: ['data/kitti_segmentation.tfrecord']})
        next_element = iterator.get_next()
        i = 1
        while i < 5:
            i += 1
            print("*"*10)
            image, label = sess.run(next_element)
            new = scipy.misc.imresize(image,(160*2,576*2) )
            plt.imshow(image[:,:,:])
            plt.show()
            plt.imshow(new[:,:,:])
#             plt.imshow(label[:,:,0], cmap='jet', alpha=0.5)
            plt.show()
            print(new.shape, image.shape)

In [4]:
# record_iterator = tf.python_io.tf_record_iterator(path='data/kitti_segmentation.tfrecord')
# string_record = next(record_iterator)
# example = tf.train.Example()
# example.ParseFromString(string_record)

## Model

In [5]:
encoder = SlimModelEncoder(name="vgg_16", num_classes=2, is_training=True)
image, label = iterator.get_next()
init_fn, end_points = encoder.build(image=image, image_shape=image_shape)

In [6]:
end_points.keys()

odict_keys(['vgg_16/conv1/conv1_1', 'vgg_16/conv1/conv1_2', 'vgg_16/pool1', 'vgg_16/conv2/conv2_1', 'vgg_16/conv2/conv2_2', 'vgg_16/pool2', 'vgg_16/conv3/conv3_1', 'vgg_16/conv3/conv3_2', 'vgg_16/conv3/conv3_3', 'vgg_16/pool3', 'vgg_16/conv4/conv4_1', 'vgg_16/conv4/conv4_2', 'vgg_16/conv4/conv4_3', 'vgg_16/pool4', 'vgg_16/conv5/conv5_1', 'vgg_16/conv5/conv5_2', 'vgg_16/conv5/conv5_3', 'vgg_16/pool5', 'vgg_16/fc6', 'vgg_16/fc7', 'vgg_16/fc8'])

In [7]:
print(end_points['vgg_16/conv3/conv3_3'], 
    end_points['vgg_16/conv4/conv4_3'], 
    end_points['vgg_16/conv5/conv5_3'],
    end_points['vgg_16/fc6'],
    end_points['vgg_16/fc7'],sep='\n')

Tensor("vgg_16/conv3/conv3_3/Relu:0", shape=(1, 40, 144, 256), dtype=float32)
Tensor("vgg_16/conv4/conv4_3/Relu:0", shape=(1, 20, 72, 512), dtype=float32)
Tensor("vgg_16/conv5/conv5_3/Relu:0", shape=(1, 10, 36, 512), dtype=float32)
Tensor("vgg_16/fc6/Relu:0", shape=(1, 5, 18, 4096), dtype=float32)
Tensor("vgg_16/fc7/Relu:0", shape=(1, 5, 18, 4096), dtype=float32)


In [8]:
decoder = FCNDecoder(end_points, nb_classes=2, scope='fcn')

In [9]:
tensors_to_connect = OrderedDict()
tensors_to_connect["vgg_16/fc7"] = (2,2)
tensors_to_connect['vgg_16/conv5/conv5_3'] = (2,2)
tensors_to_connect['vgg_16/conv4/conv4_3'] = (8,8)

In [10]:
net = decoder.build(tensors_to_connect)

## Train

In [11]:
trainer = Trainer(nb_classes=2, optimizer=tf.train.AdamOptimizer, learning_rate=5e-5)

In [ ]:
trainer.build(predictions=net, labels=label)

pred shape (92160, 2), label shape (92160, 2)


In [ ]:
trainer.train(iterator,['data/kitti_segmentation.tfrecord'])

INFO:tensorflow:Starting Session.
INFO:tensorflow:Saving checkpoint to path /tmp/tf/model.ckpt
INFO:tensorflow:Starting Queues.
INFO:tensorflow:global_step/sec: 0
INFO:tensorflow:global step 1: loss = 0.6957 (7.701 sec/step)
INFO:tensorflow:global step 2: loss = 0.6958 (2.600 sec/step)
INFO:tensorflow:Recording summary at step 2.
INFO:tensorflow:global step 3: loss = 0.6955 (2.239 sec/step)
INFO:tensorflow:global step 4: loss = 0.6952 (2.106 sec/step)
INFO:tensorflow:global step 5: loss = 0.6953 (2.081 sec/step)
INFO:tensorflow:global step 6: loss = 0.6954 (2.133 sec/step)
INFO:tensorflow:global step 7: loss = 0.6950 (2.250 sec/step)
INFO:tensorflow:global step 8: loss = 0.6951 (2.221 sec/step)
INFO:tensorflow:global step 9: loss = 0.6950 (2.196 sec/step)
INFO:tensorflow:global step 10: loss = 0.6950 (2.140 sec/step)
INFO:tensorflow:global step 11: loss = 0.6950 (2.270 sec/step)
INFO:tensorflow:global step 12: loss = 0.6947 (2.164 sec/step)
INFO:tensorflow:global step 13: loss = 0.6950

In [ ]:
tf.trainable_variables()

In [ ]:
variables = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, 'fcn')

In [ ]:
variables

In [ ]:
from tensorflow.python.framework import ops

In [ ]:
ops.get_collection(ops.GraphKeys.LOSSES)

In [ ]:
ops.get_collection(ops.GraphKeys.REGULARIZATION_LOSSES,'fcn')